In [1]:
from bs4 import BeautifulSoup, Comment
from datetime import datetime, timedelta, timezone

import requests
import pandas as pd
import time

In [4]:
dates = pd.date_range(start = "2024-03-28", end = "2024-03-28")
base_link = "https://www.basketball-reference.com"

for date in dates:
    link = base_link + "/boxscores/?month={0}&day={1}&year={2}".format(date.month, date.day, date.year)

    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")

    # Get all games for the day
    try:
        games = soup.find("div", class_ = "game_summaries").find_all("table", class_= "teams")
        # Loop through all games to get boxscore link
        for game in games:
            boxscore_link = game.find("td", class_ = "gamelink").find("a", href = True)['href']

            # Access boxscore
            page = requests.get(base_link + boxscore_link)
            time.sleep(2)
            soup = BeautifulSoup(page.content, "html.parser")

            teams = soup.find("div", class_="scorebox").find_all("strong")
            team1_text = teams[0].text.strip()
            team2_text = teams[1].text.strip()
            
            #team1 = Team.objects.get(team_abbr = team1_text)
            #team2 = Team.objects.get(team_abbr = team2_text)

            # Line Score data is stored in comments so this is a work around
            comment = soup.find("div", id = 'all_line_score').find(string=lambda text: isinstance(text, Comment))
            if "div_line_score" in comment:
                line_score = BeautifulSoup(comment.string, "html.parser")

                data = line_score.find('tbody').find_all("td")

                team1_q1_points = int(data[0].text.strip())
                team1_q2_points = int(data[1].text.strip())
                team1_q3_points = int(data[2].text.strip())
                team1_q4_points = int(data[3].text.strip())
                team1_total = int(data[4].text.strip())

                team2_q1_points = int(data[5].text.strip())
                team2_q2_points = int(data[6].text.strip())
                team2_q3_points = int(data[7].text.strip())
                team2_q4_points = int(data[8].text.strip())
                team2_total = int(data[9].text.strip())

            # Four Factors data is stored in comments
            comment = soup.find("div", id = 'all_four_factors').find(string=lambda text: isinstance(text, Comment))
            if "div_four_factors" in comment:
                factors = BeautifulSoup(comment.string, "html.parser")
                
                data = factors.find_all('td')
                team1_pace = float(data[0].text.strip())
                team1_efg = float(data[1].text.strip())
                team1_tov = float(data[2].text.strip())
                team1_orb = float(data[3].text.strip())
                team1_ft_per_fg = float(data[4].text.strip())
                team1_off_rtg = float(data[5].text.strip())

                team2_pace = float(data[6].text.strip())
                team2_efg = float(data[7].text.strip())
                team2_tov = float(data[8].text.strip())
                team2_orb = float(data[9].text.strip())
                team2_ft_per_fg = float(data[10].text.strip())
                team2_off_rtg = float(data[11].text.strip())   
            
            # box_score = BoxScore(date=date.strftime('%Y-%m-%d'),
            #                         team1 = team1,
            #                         team2 = team2,
            #                         team1_q1_points = team1_q1_points,
            #                         team1_q2_points = team1_q2_points,
            #                         team1_q3_points = team1_q3_points,
            #                         team1_q4_points = team1_q4_points,
            #                         team2_q1_points = team2_q1_points,
            #                         team2_q2_points = team2_q2_points,
            #                         team2_q3_points = team2_q3_points,
            #                         team2_q4_points = team2_q4_points,
            #                         team1_pace = team1_pace,
            #                         team1_efg = team1_efg,
            #                         team1_tov = team1_tov,
            #                         team1_orb = team1_orb,
            #                         team1_ft_per_fg = team1_ft_per_fg,
            #                         team1_off_rtg = team1_off_rtg,
            #                         team2_pace = team2_pace,
            #                         team2_efg = team2_efg,
            #                         team2_tov = team2_tov,
            #                         team2_orb = team2_orb,
            #                         team2_ft_per_fg = team2_ft_per_fg,
            #                         team2_off_rtg = team2_off_rtg)
            # box_score.save()
            print(f"BOX SCORE FOR {team1_text} vs {team2_text} ON {date.month}-{date.day}-{date.year} ADDED!")
    except Exception as e:
        print(e)
        print(f"NO GAMES PLAYED ON {date.month}-{date.day}-{date.year}.")
    

BOX SCORE FOR Boston Celtics vs Cleveland Cavaliers ON 5-13-2024 ADDED!
BOX SCORE FOR Oklahoma City Thunder vs Dallas Mavericks ON 5-13-2024 ADDED!


In [2]:
today = datetime.today()
tomorrow = today + timedelta(days=1)

In [3]:
today = today.strftime("%Y-%m-%dT00:00:00Z")
tomorrow = tomorrow.strftime("%Y-%m-%dT06:00:00Z")
print(today, tomorrow)

2024-05-14T00:00:00Z 2024-05-15T06:00:00Z


In [4]:
resp = requests.get("https://api.the-odds-api.com/v4/sports/basketball_nba/odds",
                    params={
                        "api_key" : "18d4f5a57823e4304f11070456e9b30e",
                        "regions" : "us",
                        "markets" : "h2h,spreads,totals",
                        "oddsFormat" : "american",
                        "dateFormat" : "iso",
                        "bookmakers" : "fanduel",
                        "commenceTimeFrom" : today,
                        "commenceTimeTo" : tomorrow
                    })

In [4]:
resp = requests.get("https://api.the-odds-api.com/v4/sports/basketball_nba/events/63997e6c1cd71cb56e14a16799fa3334/odds?",
                    params={
                        "api_key" : "18d4f5a57823e4304f11070456e9b30e",
                        "regions" : "us",
                        "markets" : "player_points,player_rebounds,player_assists,player_threes,player_points_rebounds_assists,player_points_assists",
                        "oddsFormat" : "american",
                        "dateFormat" : "iso",
                        "bookmakers" : "fanduel",
                        "commenceTimeFrom" : today,
                        "commenceTimeTo" : tomorrow
                    })

In [5]:
if resp.status_code != 200:
    print(f'Failed to get odds: status_code {resp.status_code}, response body {resp.text}')

else:
    odds_json = resp.json()
    print('Number of events:', len(odds_json))
    # Check the usage quota
    print('Remaining requests', resp.headers['x-requests-remaining'])
    print('Used requests', resp.headers['x-requests-used'])

Number of events: 7
Remaining requests 302
Used requests 198


In [9]:
resp.text

'{"id":"63997e6c1cd71cb56e14a16799fa3334","sport_key":"basketball_nba","sport_title":"NBA","commence_time":"2024-05-13T23:00:00Z","home_team":"Cleveland Cavaliers","away_team":"Boston Celtics","bookmakers":[{"key":"fanduel","title":"FanDuel","markets":[{"key":"player_assists","last_update":"2024-05-13T19:59:22Z","outcomes":[{"name":"Over","description":"Caris LeVert","price":-144,"point":2.5},{"name":"Under","description":"Caris LeVert","price":118,"point":2.5},{"name":"Over","description":"Jaylen Brown","price":-142,"point":2.5},{"name":"Under","description":"Jaylen Brown","price":116,"point":2.5},{"name":"Over","description":"Al Horford","price":-112,"point":2.5},{"name":"Under","description":"Al Horford","price":-108,"point":2.5},{"name":"Over","description":"Max Strus","price":114,"point":2.5},{"name":"Under","description":"Max Strus","price":-140,"point":2.5},{"name":"Over","description":"Donovan Mitchell","price":126,"point":5.5},{"name":"Under","description":"Donovan Mitchell","

In [6]:
import json

print(json.dumps(resp.json(), indent=2))

{
  "id": "6915923f8082157d34fb1c2a9a9fe60a",
  "sport_key": "basketball_nba",
  "sport_title": "NBA",
  "commence_time": "2024-05-15T00:05:00Z",
  "home_team": "New York Knicks",
  "away_team": "Indiana Pacers",
  "bookmakers": [
    {
      "key": "fanduel",
      "title": "FanDuel",
      "markets": [
        {
          "key": "player_points",
          "last_update": "2024-05-14T17:04:29Z",
          "outcomes": [
            {
              "name": "Over",
              "description": "Isaiah Hartenstein",
              "price": -120,
              "point": 9.5
            },
            {
              "name": "Under",
              "description": "Isaiah Hartenstein",
              "price": -106,
              "point": 9.5
            },
            {
              "name": "Over",
              "description": "Andrew Nembhard",
              "price": -111,
              "point": 9.5
            },
            {
              "name": "Under",
              "description": "Andrew

In [46]:
players = {}

In [47]:
for bm in resp.json()['bookmakers']:
    for m in bm['markets']:
        for o in m['outcomes']:
            if o['description'] not in players:
                players[o['description']] = {"over" : {}, "under" : {}}
                players[o['description']]['last_update'] = m['last_update']

            if o['name'] == "Over":
                players[o['description']]["over"][m['key']] = (o['price'], o['point'])  
            else:
                players[o['description']]["under"][m['key']] = (o['price'], o['point'])                                                    

In [48]:
players

{'Isaiah Hartenstein': {'over': {'player_points': (-120, 9.5),
   'player_rebounds': (-130, 9.5)},
  'under': {'player_points': (-106, 9.5), 'player_rebounds': (102, 9.5)},
  'last_update': '2024-05-14T17:04:29Z'},
 'Andrew Nembhard': {'over': {'player_points': (-111, 9.5),
   'player_rebounds': (128, 2.5)},
  'under': {'player_points': (-115, 9.5), 'player_rebounds': (-164, 2.5)},
  'last_update': '2024-05-14T17:04:29Z'},
 'Ben Sheppard': {'over': {'player_points': (100, 4.5)},
  'under': {'player_points': (-128, 4.5)},
  'last_update': '2024-05-14T17:04:29Z'},
 'Donte DiVincenzo': {'over': {'player_points': (-118, 22.5),
   'player_rebounds': (-146, 4.5)},
  'under': {'player_points': (-108, 22.5), 'player_rebounds': (114, 4.5)},
  'last_update': '2024-05-14T17:04:29Z'},
 'Tyrese Haliburton': {'over': {'player_points': (-120, 20.5),
   'player_rebounds': (-104, 4.5)},
  'under': {'player_points': (-106, 20.5), 'player_rebounds': (-122, 4.5)},
  'last_update': '2024-05-14T17:04:29Z'},

In [64]:
for p,v in players.items():
    print(p, v['last_update']) 
    for over_under, over_under_v in v.items():
        if over_under == "over":
            print("OVER")
            for prop, prop_v in over_under_v.items():
                if (prop == "player_points"):
                    print("PLAYER POINTS:", prop_v[0], prop_v[1])
        if over_under == "under": 
            print("UNDER")
            for prop, prop_v in over_under_v.items():
                print(prop, prop_v)                                                  

Isaiah Hartenstein 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: -120 9.5
UNDER
player_points (-106, 9.5)
player_rebounds (102, 9.5)
Andrew Nembhard 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: -111 9.5
UNDER
player_points (-115, 9.5)
player_rebounds (-164, 2.5)
Ben Sheppard 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: 100 4.5
UNDER
player_points (-128, 4.5)
Donte DiVincenzo 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: -118 22.5
UNDER
player_points (-108, 22.5)
player_rebounds (114, 4.5)
Tyrese Haliburton 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: -120 20.5
UNDER
player_points (-106, 20.5)
player_rebounds (-122, 4.5)
Miles McBride 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: -118 9.5
UNDER
player_points (-108, 9.5)
Alec Burks 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: -108 9.5
UNDER
player_points (-118, 9.5)
Obi Toppin 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: -130 8.5
UNDER
player_points (102, 8.5)
player_rebounds (-128, 3.5)
T.J. McConnell 2024-05-14T17:04:29Z
OVER
PLAYER POINTS: -113 10.5
UNDER
player_points

In [62]:
est_offset = timedelta(hours=-6)
original_timezone = timezone.utc

In [76]:
for json in odds_json:
    home_team = json['home_team']
    away_team = json['away_team']
    time = datetime.fromisoformat(json['commence_time']).astimezone(original_timezone) + est_offset
    bm = json['bookmakers']

    # Will need to change if more sports books are added
    bookmaker = bm[0]['title']
    markets = bm[0]['markets']

    for m in markets:
        key = m['key']
        for team in m['outcomes']:
            if team['name'] == home_team:
                if key == 'h2h':
                    home_h2h_price = team['price']
                elif key == 'spreads':
                    home_spreads_price = team['price']
                    home_spreads_points = team['point']
            elif team['name'] == away_team:
                if key == 'h2h':
                    away_h2h_price = team['price']
                elif key == 'spreads':
                    away_spreads_price = team['price']
                    away_spreads_points = team['point']
            elif team['name'] == "Over":
                over_price = team['price']
                over_point = team['point']
            else:
                under_price = team['price']
                under_point = team['point']


    print(time)
    print(home_team, home_h2h_price, home_spreads_price, home_spreads_points)
    print(away_team, away_h2h_price, away_spreads_price, away_spreads_points)
    print(over_price, over_point)
    print(under_price, under_point)
    print()



2024-05-09 17:00:00+00:00
Boston Celtics -950 -112 -13.5
Cleveland Cavaliers 640 -108 13.5
-110 212.5
-110 212.5

2024-05-09 19:30:00+00:00
Oklahoma City Thunder -200 -110 -5.0
Dallas Mavericks 168 -110 5.0
-108 218.0
-112 218.0



In [29]:
import json

print(json.dumps(odds_json, indent=4))

{
    "id": "b26581715ce0b9f12463cd2e3317516f",
    "sport_key": "basketball_nba",
    "sport_title": "NBA",
    "commence_time": "2024-05-10T23:10:00Z",
    "home_team": "Indiana Pacers",
    "away_team": "New York Knicks",
    "bookmakers": [
        {
            "key": "fanduel",
            "title": "FanDuel",
            "markets": [
                {
                    "key": "player_assists",
                    "last_update": "2024-05-10T23:06:53Z",
                    "outcomes": [
                        {
                            "name": "Over",
                            "description": "Tyrese Haliburton",
                            "price": -104,
                            "point": 9.5
                        },
                        {
                            "name": "Under",
                            "description": "Tyrese Haliburton",
                            "price": -118,
                            "point": 9.5
                        },
           

In [124]:
base_link = "https://www.basketball-reference.com"

page = requests.get(base_link + "/teams")
soup = BeautifulSoup(page.content, "html.parser")

teams = soup.find("table", id = "teams_active").find_all("tr", class_ = "full_table")
for team in teams:
    print(team)
    page = requests.get(base_link + team.find("a", href=True)['href'])
    soup = BeautifulSoup(page.content, "html.parser")
    time.sleep(2)
    
    # Get most recent team
    team_name = soup.find("h1").text.strip()
    rows = soup.find("tbody").find_all("tr")
    for row in rows:
        cols = row.find_all('td')
        team_link = cols[1].find("a", href=True)['href']

        page = requests.get(base_link + team_link)
        soup = BeautifulSoup(page.content, "html.parser")
        time.sleep(2)

        # Get all players on Roster
        players_table = soup.find("table", id = "roster")
        for row in players_table.tbody.find_all("tr"):
            number = int(row.find("th").text.strip())
            cols = row.find_all('td')

            if (cols != []):
                name = cols[0].text.strip()
                position = cols[1].text.strip()

                #Check for two way player identifier
                if(name[-1:] == ")"):
                    name = name.replace("(TW)", "")

                print(team_name, number, name, position)
        print()
        break
        



    

<tr class="full_table"><th class="left" data-stat="franch_name" scope="row"><a href="/teams/ATL/">Atlanta Hawks</a></th><td class="left" data-stat="lg_id">NBA</td><td class="right" data-stat="year_min">1949-50</td><td class="right" data-stat="year_max">2023-24</td><td class="right" data-stat="years">75</td><td class="right" data-stat="g">5937</td><td class="right" data-stat="wins">2927</td><td class="right" data-stat="losses">3010</td><td class="right" data-stat="win_loss_pct">.493</td><td class="right" data-stat="years_playoffs">49</td><td class="right" data-stat="years_division_champion">12</td><td class="right iz" data-stat="years_conference_champion">0</td><td class="right" data-stat="years_league_champion">1</td></tr>


KeyboardInterrupt: 